# **MÓDULO 35 - Cross Validation**

Nesta tarefa, você trabalhará com uma base de dados que contém informações sobre variáveis ambientais coletadas para a detecção de incêndios. O objetivo é utilizar técnicas de validação cruzada (cross-validation) para avaliar a performance de um modelo de classificação na previsão da ocorrência de um incêndio com base nas variáveis fornecidas.


Descrição da Base de Dados
A base de dados contém as seguintes variáveis:

Unnamed:0: Índice (não é uma variável útil para o modelo)

UTC: Tempo em Segundos UTC

Temperature[C]: Temperatura do Ar (em graus Celsius)

Humidity[%]: Umidade do Ar (em porcentagem)

TVOC[ppb]: Total de Compostos Orgânicos Voláteis (medido em partes por bilhão)

eCO2[ppm]: Concentração equivalente de CO2 (medido em partes por milhão)

Raw H2: Hidrogênio molecular bruto, não compensado

Raw Ethanol: Etanol gasoso bruto

Pressure[hPA]: Pressão do Ar (em hectopascais)

PM1.0: Material particulado de tamanho < 1,0 µm

PM2.5: Material particulado de tamanho >1,0 µm e < 2,5 µm

NC0.5: Concentração numérica de material particulado de tamanho < 0,5 µm

NC1.0: Concentração numérica de material particulado de tamanho 0,5 µm < 1,0 µm

NC2.5: Concentração numérica de material particulado de tamanho 1,0 µm < 2,5 µm

CNT: Contador de amostras


E a variável alvo:

Fire Alarm: Indicador binário de incêndio (1 se houver incêndio, 0 caso contrário)

O objetivo desta tarefa é aplicar a técnica de validação cruzada (cross-validation) para avaliar a performance de um modelo de classificação. A validação cruzada ajudará a garantir que o modelo seja avaliado de maneira robusta e generalize bem para dados não vistos.

In [2]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

# 1 - Carregue a base de dados, verifique os tipos de dados e também se há presença de dados faltantes ou nulos.

In [5]:
# seu código aqui
df = pd.read_csv('/content/Cientista de dados M35 - smoke_detection_iot.csv')
df.head(10)

,Unnamed: 0,UTC,Temperature[C],Humidity[%],TVOC[ppb],eCO2[ppm],Raw H2,Raw Ethanol,Pressure[hPa],PM1.0,PM2.5,NC0.5,NC1.0,NC2.5,CNT,Fire Alarm
0,0,1654733331,20.000,57.36,0,400,12306,18520,939.735,0.0,0.00,0.0,0.000,0.00,0,0
1,1,1654733332,20.015,56.67,0,400,12345,18651,939.744,0.0,0.00,0.0,0.000,0.00,1,0
2,2,1654733333,20.029,55.96,0,400,12374,18764,939.738,0.0,0.00,0.0,0.000,0.00,2,0
3,3,1654733334,20.044,55.28,0,400,12390,18849,939.736,0.0,0.00,0.0,0.000,0.00,3,0
4,4,1654733335,20.059,54.69,0,400,12403,18921,939.744,0.0,0.00,0.0,0.000,0.00,4,0
5,5,1654733336,20.073,54.12,0,400,12419,18998,939.725,0.0,0.00,0.0,0.000,0.00,5,0
6,6,1654733337,20.088,53.61,0,400,12432,19058,939.738,0.0,0.00,0.0,0.000,0.00,6,0
7,7,1654733338,20.103,53.20,0,400,12439,19114,939.758,0.0,0.00,0.0,0.000,0.00,7,0
8,8,1654733339,20.117,52.81,0,400,12448,19155,939.758,0.0,0.00,0.0,0.000,0.00,8,0
9,9,1654733340,20.132,52.46,0,400,12453,19195,939.756,0.9,3.78,0.0,4.369,2.78,9,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62630 entries, 0 to 62629
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      62630 non-null  int64  
 1   UTC             62630 non-null  int64  
 2   Temperature[C]  62630 non-null  float64
 3   Humidity[%]     62630 non-null  float64
 4   TVOC[ppb]       62630 non-null  int64  
 5   eCO2[ppm]       62630 non-null  int64  
 6   Raw H2          62630 non-null  int64  
 7   Raw Ethanol     62630 non-null  int64  
 8   Pressure[hPa]   62630 non-null  float64
 9   PM1.0           62630 non-null  float64
 10  PM2.5           62630 non-null  float64
 11  NC0.5           62630 non-null  float64
 12  NC1.0           62630 non-null  float64
 13  NC2.5           62630 non-null  float64
 14  CNT             62630 non-null  int64  
 15  Fire Alarm      62630 non-null  int64  
dtypes: float64(8), int64(8)
memory usage: 7.6 MB


###Transformação do Dtype de Fire Alarm:

In [10]:
df['Fire_Alarm'] = df['Fire_Alarm'].astype('category')

##Verificação de dados nulos:

In [7]:
(df.isnull().sum() / df.shape[0]) *100

,0
Unnamed: 0,0.0
UTC,0.0
Temperature[C],0.0
Humidity[%],0.0
TVOC[ppb],0.0
eCO2[ppm],0.0
Raw H2,0.0
Raw Ethanol,0.0
Pressure[hPa],0.0
PM1.0,0.0


In [8]:
df.rename(columns={'Fire Alarm': 'Fire_Alarm'}, inplace=True)

Para a coluna Fire Alarm, por conta do espaçamento talvez seja util renomear o nome da coluna utilizando:

df.rename(columns={'Fire Alarm': 'Fire_Alarm'}, inplace=True)

###Acredito que não faça sentido para uso manter a coluna UTC, me parece uma coluna de identificação

In [11]:
df.drop(columns=['UTC'], inplace=True)

# 2 - Para essa base, onde você realizará as previsões de fire alarm, qual modelo de machine learning você aplicará? Justifique.




Para esta base faria a utilização do modelo de regressão logística, que se porta melhor na previsão de eventos categóricos, que é com o que estamos lidando aqui

# 3 - Separe a base em Y e X e já rode a instância do modelo que você utilizará.

Separação da base:

In [13]:
# seu código aqui
X = df.drop('Fire_Alarm', axis=1)
y = df['Fire_Alarm']

Tamanho da base separada:

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("\nTamanhos das bases após a separação:")
print(f"X_train: {X_train.shape}")
print(f"X_test: {X_test.shape}")
print(f"y_train: {y_train.shape}")
print(f"y_test: {y_test.shape}")


Tamanhos das bases após a separação:
X_train: (50104, 14)
X_test: (12526, 14)
y_train: (50104,)
y_test: (12526,)


In [15]:
model = LogisticRegression(random_state=42, solver='liblinear')

# Treinar o modelo
model.fit(X_train, y_train)

LogisticRegression(random_state=42, solver='liblinear')

In [23]:
y_pred = model.predict(X_test)

# Calcular métricas
accuracy_test = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=['No Fire (0)', 'Fire (1)'])

In [24]:
print(f"Acurácia no Conjunto de Teste: {accuracy_test:.4f}")

Acurácia no Conjunto de Teste: 0.9850


In [25]:
print(report)

              precision    recall  f1-score   support

 No Fire (0)       0.96      0.99      0.97      3575
    Fire (1)       1.00      0.98      0.99      8951

    accuracy                           0.98     12526
   macro avg       0.98      0.99      0.98     12526
weighted avg       0.99      0.98      0.99     12526



Temos métricas alcançando valores altos e isto pode indicar overfitting já que utilizamos regressão logística como modelo para nossa base, farei a aplicação do cross validation e acredito que a acurácia diminua

# 4 - Defina o número de Folds e rode o modelo com a validação cruzada.

In [16]:
# seu código aqui
K_FOLDS = 10

Optei por utilizar 10 folds para abranger mais as possibilidades, dessa forma fazendo diferentes testes dentro da base de forma mais separada

In [17]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

scores = cross_val_score(model, X, y, cv=kf, scoring='accuracy')

Optei por utilizar de métrica avaliativa a acurácia para que pudesse ter melhor aproveitamento da média de acertos do modelo

In [18]:
scores = cross_val_score(
    estimator=model,
    X=X,
    y=y,
    cv=K_FOLDS,
    scoring='accuracy',
    n_jobs=-1
)

# 5 - Avalie a pontuação de cada modelo e ao final a validação final da média.

In [21]:
import numpy as np
print(f"Métricas (Acurácia) de cada um dos {K_FOLDS} folds:")
print(np.round(scores, 4))
print(f"\nAcurácia Média: {scores.mean():.4f}")

Métricas (Acurácia) de cada um dos 10 folds:
[0.817  0.8218 1.     0.9483 0.9106 0.9941 0.9682 1.     0.9994 0.8438]

Acurácia Média: 0.9303


In [26]:
print(report)

              precision    recall  f1-score   support

 No Fire (0)       0.96      0.99      0.97      3575
    Fire (1)       1.00      0.98      0.99      8951

    accuracy                           0.98     12526
   macro avg       0.98      0.99      0.98     12526
weighted avg       0.99      0.98      0.99     12526



Como esperado a acurácia diminuiu após ser utilizado o cross validation, isto indica que havia sim a possibilidade de overfitting, mas também indica que nosso modelo se encaixa muito bem para os dados capaz de prever bem os acontecimentos.